# Análisis Exploratorio de Datos - INE Guatemala

**Estadísticas Vitales: Nacimientos, Defunciones, Matrimonios, Divorcios y Defunciones Fetales**

Datos del Instituto Nacional de Estadística (INE) de Guatemala, período 2012-2022.

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as sp_stats
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

from src.export import to_dataframe, list_collections, collection_stats, get_column_labels

plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100
sns.set_theme(style='whitegrid')

import warnings
warnings.filterwarnings('ignore', message='.*Creating legend with loc.*')
warnings.filterwarnings('ignore', message='.*Tight layout.*')

print('Setup completo')

## 1. Descripción General del Conjunto de Datos

### 1.1 Colecciones disponibles

In [ ]:
# Ver todas las colecciones y sus estadísticas
col_stats = collection_stats()
for name, data in col_stats.items():
    if name.startswith('_'):
        continue
    print(f"{name:25s}: {data['count']:>9,} documentos | {len(data['fields']):>3} campos")
    print(f"  Campos: {', '.join(data['fields'][:15])}")
    print()

### 1.2 Cargar dataset principal

In [ ]:
# Elegir el dataset a analizar
# Opciones: 'nacimientos', 'defunciones', 'matrimonios', 'divorcios', 'defunciones_fetales'
DATASET = 'nacimientos'  # <-- CAMBIAR AQUÍ

df = to_dataframe(DATASET)
labels_map = get_column_labels(DATASET)

print(f"Dataset: {DATASET}")
print(f"Filas: {len(df):,}")
print(f"Columnas: {len(df.columns)}")
print(f"Años disponibles: {sorted(df['_year'].dropna().unique().astype(int))}")

In [ ]:
# Descripción de cada variable con su label descriptivo
meta_cols = ['_year', '_source_file']
analysis_cols = [c for c in df.columns if c not in meta_cols]

var_info = []
for col in analysis_cols:
    desc = labels_map.get(col, col)
    dtype = str(df[col].dtype)
    n_unique = df[col].nunique()
    n_null = df[col].isnull().sum()
    pct_null = round(n_null / len(df) * 100, 1)
    var_info.append({
        'columna': col,
        'descripcion': desc,
        'tipo': dtype,
        'valores_unicos': n_unique,
        'nulos': n_null,
        'pct_nulos': pct_null
    })

var_df = pd.DataFrame(var_info)
print(f"Total variables de análisis: {len(var_df)}")
var_df

In [ ]:
# Descripción de cada variable con su label descriptivo
meta_cols = ['_year', '_source_file']
analysis_cols = [c for c in df.columns if c not in meta_cols]

var_info = []
for col in analysis_cols:
    desc = labels_map.get(col, col)
    dtype = str(df[col].dtype)
    n_unique = df[col].nunique()
    n_null = df[col].isnull().sum()
    pct_null = round(n_null / len(df) * 100, 1)
    var_info.append({
        'columna': col,
        'descripcion': desc,
        'tipo': dtype,
        'valores_unicos': n_unique,
        'nulos': n_null,
        'pct_nulos': pct_null
    })

var_df = pd.DataFrame(var_info)
print(f"Total variables de análisis: {len(var_df)}")
var_df

In [ ]:
# Clasificar variables en numéricas y categóricas
# Intentar convertir columnas que parecen numéricas pero están como string
for col in analysis_cols:
    if df[col].dtype == 'object' or str(df[col].dtype).startswith('str'):
        try:
            converted = pd.to_numeric(df[col], errors='coerce')
            if converted.notna().sum() / df[col].notna().sum() > 0.8:
                df[col] = converted
        except Exception:
            pass

num_cols = df[analysis_cols].select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in analysis_cols if c not in num_cols]

def label(col, max_len=40):
    """Retorna el label descriptivo, truncado y sin notas."""
    raw = labels_map.get(col, col)
    if 'Nota:' in raw:
        raw = raw.split('Nota:')[0].strip()
    if len(raw) > max_len:
        raw = raw[:max_len-3] + '...'
    return raw

print(f"Variables numéricas ({len(num_cols)}):")
for c in num_cols:
    print(f"  {c:15s} -> {label(c)}")

print(f"\nVariables categóricas ({len(cat_cols)}):")
for c in cat_cols:
    print(f"  {c:15s} -> {label(c)} ({df[c].nunique()} cat.)")

## 2. Exploración de Variables Numéricas

### 2.1 Estadísticas descriptivas

In [ ]:
# Excluir variables que son solo identificadores (año de registro, año de ocurrencia)
id_cols = [c for c in num_cols if 'reg' in c or (df[c].dropna().between(2000, 2030).mean() > 0.9)]
meaningful_num = [c for c in num_cols if c not in id_cols]

print(f"Variables numéricas de análisis (excluyendo años/IDs): {len(meaningful_num)}")
for c in meaningful_num:
    print(f"  {c}: {label(c)}")

if id_cols:
    print(f"\nExcluidas (años/IDs): {[f'{c} ({label(c)})' for c in id_cols]}")

In [ ]:
# Medidas de tendencia central, dispersión y orden
if meaningful_num:
    desc = df[meaningful_num].describe().T
    desc['mediana'] = df[meaningful_num].median()
    desc['moda'] = df[meaningful_num].mode().iloc[0] if len(df[meaningful_num].mode()) > 0 else None
    desc['asimetria'] = df[meaningful_num].skew()
    desc['curtosis'] = df[meaningful_num].kurtosis()
    desc['cv_%'] = (desc['std'] / desc['mean'] * 100).round(2)
    desc['riq'] = desc['75%'] - desc['25%']
    desc.index = [f"{c} ({label(c)})" for c in desc.index]
    desc
else:
    print("No hay variables numéricas significativas (las existentes son solo años/IDs)")
    print("Esto es normal para este dataset donde la mayoría de variables son categóricas.")
    desc = None

### 2.2 Histogramas

In [ ]:
# Histogramas de variables numéricas significativas
plot_num = [c for c in meaningful_num if df[c].nunique() > 2]

if plot_num:
    n = len(plot_num)
    ncols_plot = min(3, n)
    nrows_plot = (n + ncols_plot - 1) // ncols_plot
    fig, axes = plt.subplots(nrows_plot, ncols_plot, figsize=(5*ncols_plot, 4*nrows_plot))
    axes = np.array(axes).flatten() if n > 1 else [axes]

    for i, col in enumerate(plot_num):
        ax = axes[i]
        data = df[col].dropna()
        ax.hist(data, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
        ax.set_title(label(col), fontsize=11)
        ax.set_xlabel(col)
        ax.set_ylabel('Frecuencia')

    for j in range(i+1, len(axes)):
        axes[j].set_visible(False)

    plt.suptitle('Distribución de Variables Numéricas', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()
else:
    print("No hay variables numéricas significativas para graficar histogramas.")

### 2.3 Boxplots

In [ ]:
if plot_num:
    n = len(plot_num)
    ncols_plot = min(3, n)
    nrows_plot = (n + ncols_plot - 1) // ncols_plot
    fig, axes = plt.subplots(nrows_plot, ncols_plot, figsize=(5*ncols_plot, 4*nrows_plot))
    axes = np.array(axes).flatten() if n > 1 else [axes]

    for i, col in enumerate(plot_num):
        ax = axes[i]
        data = df[col].dropna()
        ax.boxplot(data, vert=True)
        ax.set_title(label(col), fontsize=11)
        ax.set_ylabel(col)

    for j in range(i+1, len(axes)):
        axes[j].set_visible(False)

    plt.suptitle('Boxplots de Variables Numéricas', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()
else:
    print("No hay variables numéricas significativas para graficar boxplots.")

### 2.4 Test de normalidad (Shapiro-Wilk)

Se usa una muestra de 5000 observaciones (límite de Shapiro-Wilk).

In [ ]:
# Test de normalidad para variables numéricas significativas
normality_results = []
sample_size = 5000

test_cols = meaningful_num if meaningful_num else num_cols

for col in test_cols:
    data = df[col].dropna()
    if len(data) < 3:
        continue
    sample = data.sample(min(sample_size, len(data)), random_state=42)
    stat, p_value = sp_stats.shapiro(sample)
    normality_results.append({
        'variable': col,
        'descripcion': label(col),
        'statistic': round(stat, 4),
        'p_value': f"{p_value:.2e}",
        'normal (α=0.05)': 'Sí' if p_value > 0.05 else 'No',
        'asimetria': round(data.skew(), 3),
    })

norm_df = pd.DataFrame(normality_results)
if len(norm_df) > 0:
    print("Resultados del test de Shapiro-Wilk:")
    display(norm_df)
else:
    print("No hay suficientes variables numéricas para el test de normalidad.")

## 3. Exploración de Variables Categóricas

### 3.1 Tablas de frecuencia

In [ ]:
# Tablas de frecuencia para variables categóricas principales
# Excluir municipios y variables con demasiadas categorías (>50)
main_cat_cols = [c for c in cat_cols if df[c].nunique() <= 50 and df[c].notna().sum() > len(df) * 0.1]

for col in main_cat_cols:
    desc_label = label(col)
    print(f"\n{'='*60}")
    print(f"{col} - {desc_label} ({df[col].nunique()} categorías)")
    print('='*60)

    freq = df[col].value_counts()
    prop = df[col].value_counts(normalize=True) * 100

    table = pd.DataFrame({
        'frecuencia': freq,
        'porcentaje': prop.round(2),
        'acumulado_%': prop.cumsum().round(2)
    })

    if len(table) > 20:
        print(table.head(20))
        print(f"  ... y {len(table) - 20} categorías más")
    else:
        print(table)

### 3.2 Gráficos de barras

In [ ]:
# Gráficos de barras para variables categóricas con pocas categorías
bar_cols = [c for c in main_cat_cols if 2 <= df[c].nunique() <= 25]

for col in bar_cols[:10]:  # Máximo 10 gráficos
    fig, ax = plt.subplots(figsize=(10, max(4, df[col].nunique() * 0.4)))
    freq = df[col].value_counts().head(20)

    colors = sns.color_palette('viridis', len(freq))
    freq.plot(kind='barh', ax=ax, color=colors)
    ax.set_title(f"{label(col)} ({col})", fontsize=13)
    ax.set_xlabel('Frecuencia')
    ax.set_ylabel('')

    # Agregar porcentajes
    total = freq.sum()
    for i, (val, count) in enumerate(freq.items()):
        ax.text(count + total*0.005, i, f" {count/total*100:.1f}%", va='center', fontsize=9)

    plt.tight_layout()
    plt.show()

## 4. Relaciones entre Variables

### 4.1 Matriz de correlación (variables numéricas)

In [ ]:
# Matriz de correlación con todas las numéricas
corr_cols = [c for c in num_cols if df[c].nunique() > 2]

if len(corr_cols) >= 2:
    corr_matrix = df[corr_cols].corr()

    # Labels cortos para el heatmap
    corr_labels = {c: label(c, max_len=25) for c in corr_cols}
    corr_display = corr_matrix.rename(index=corr_labels, columns=corr_labels)

    size = max(8, len(corr_cols) * 2.5)
    fig, ax = plt.subplots(figsize=(size, size * 0.8))
    mask = np.triu(np.ones_like(corr_display, dtype=bool))
    sns.heatmap(corr_display, mask=mask, annot=True, fmt='.2f',
                cmap='coolwarm', center=0, ax=ax, vmin=-1, vmax=1,
                annot_kws={'size': 10})
    ax.set_title('Matriz de Correlación', fontsize=14)
    ax.tick_params(axis='x', rotation=45, labelsize=9)
    ax.tick_params(axis='y', rotation=0, labelsize=9)
    plt.subplots_adjust(left=0.25, bottom=0.25)
    plt.show()

    # Top correlaciones
    pairs = []
    for i in range(len(corr_cols)):
        for j in range(i+1, len(corr_cols)):
            pairs.append({
                'var1': f"{corr_cols[i]} ({label(corr_cols[i])})",
                'var2': f"{corr_cols[j]} ({label(corr_cols[j])})",
                'correlacion': round(corr_matrix.iloc[i, j], 4)
            })
    pairs_df = pd.DataFrame(pairs).sort_values('correlacion', key=abs, ascending=False)
    print("\nCorrelaciones ordenadas por magnitud:")
    display(pairs_df)
else:
    print("No hay suficientes variables numéricas para la matriz de correlación.")

### 4.2 Scatter plots

In [ ]:
# Scatter plots entre pares de variables numéricas significativas
scatter_cols = meaningful_num if len(meaningful_num) >= 2 else corr_cols

if len(scatter_cols) >= 2:
    from itertools import combinations
    pairs = list(combinations(scatter_cols, 2))[:6]  # Max 6 scatter plots

    n = len(pairs)
    ncols_plot = min(3, n)
    nrows_plot = (n + ncols_plot - 1) // ncols_plot
    fig, axes = plt.subplots(nrows_plot, ncols_plot, figsize=(5*ncols_plot, 5*nrows_plot))
    axes = np.array(axes).flatten() if n > 1 else [axes]

    sample = df.sample(min(5000, len(df)), random_state=42)
    for i, (c1, c2) in enumerate(pairs):
        axes[i].scatter(sample[c1], sample[c2], alpha=0.3, s=10, color='steelblue')
        axes[i].set_xlabel(label(c1))
        axes[i].set_ylabel(label(c2))
        r = df[[c1, c2]].dropna().corr().iloc[0, 1]
        axes[i].set_title(f'{label(c1)} vs {label(c2)}\n(r={r:.3f})', fontsize=10)

    for j in range(len(pairs), len(axes)):
        axes[j].set_visible(False)

    plt.tight_layout()
    plt.show()
else:
    print("No hay suficientes variables numéricas para scatter plots.")

### 4.3 Tablas cruzadas (Crosstabs)

In [ ]:
# Crosstabs entre variables categóricas con pocas categorías
cross_candidates = [c for c in cat_cols if 2 <= df[c].nunique() <= 10 and df[c].notna().sum() > len(df) * 0.5]

if len(cross_candidates) >= 2:
    from itertools import combinations
    cross_pairs = list(combinations(cross_candidates[:6], 2))[:4]  # Top 4 cruces

    for c1, c2 in cross_pairs:
        ct = pd.crosstab(df[c1], df[c2])

        fig, ax = plt.subplots(figsize=(max(8, ct.shape[1]*1.5), max(5, ct.shape[0]*0.6)))
        sns.heatmap(ct, annot=True, fmt=',d', cmap='YlOrRd', ax=ax)
        ax.set_title(f'{label(c1)} vs {label(c2)}', fontsize=13)
        ax.set_xlabel(label(c2))
        ax.set_ylabel(label(c1))
        plt.tight_layout()
        plt.show()
else:
    print("No hay suficientes variables categóricas con pocas categorías para crosstabs.")

## 5. Análisis temporal

In [ ]:
# Registros por año
yearly = df.groupby('_year').size().sort_index()

fig, ax = plt.subplots(figsize=(10, 5))
bars = ax.bar(yearly.index.astype(int).astype(str), yearly.values,
              color='steelblue', edgecolor='black')
ax.set_title(f'Registros de {DATASET} por año', fontsize=14)
ax.set_xlabel('Año')
ax.set_ylabel('Cantidad de registros')
ax.tick_params(axis='x', rotation=45)

# Mostrar valores encima de las barras
for bar, val in zip(bars, yearly.values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + yearly.max()*0.01,
            f'{val:,}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# Distribución por mes de ocurrencia (si existe)
mes_col = None
for c in cat_cols:
    if 'mesocu' in c:
        mes_col = c
        break

if mes_col:
    monthly = df[mes_col].value_counts().sort_index()
    fig, ax = plt.subplots(figsize=(10, 5))
    monthly.plot(kind='bar', ax=ax, color='darkorange', edgecolor='black')
    ax.set_title(f'{label(mes_col)} - Distribución mensual', fontsize=14)
    ax.set_xlabel('Mes')
    ax.set_ylabel('Frecuencia')
    ax.tick_params(axis='x', rotation=45)
    plt.tight_layout()
    plt.show()

## 6. Datos atípicos (Outliers)

In [ ]:
# Detección de outliers con IQR para variables numéricas significativas
outlier_cols = meaningful_num if meaningful_num else [c for c in num_cols if df[c].nunique() > 5]

if outlier_cols:
    outlier_report = []
    for col in outlier_cols:
        data = df[col].dropna()
        Q1 = data.quantile(0.25)
        Q3 = data.quantile(0.75)
        IQR = Q3 - Q1
        if IQR == 0:
            continue
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        n_outliers = ((data < lower) | (data > upper)).sum()
        outlier_report.append({
            'variable': col,
            'descripcion': label(col),
            'Q1': Q1,
            'Q3': Q3,
            'IQR': IQR,
            'limite_inf': lower,
            'limite_sup': upper,
            'n_outliers': n_outliers,
            'pct_outliers': round(n_outliers / len(data) * 100, 2),
        })

    if outlier_report:
        outlier_df = pd.DataFrame(outlier_report)
        display(outlier_df)
    else:
        print("No se detectaron outliers con el método IQR.")
else:
    print("No hay variables numéricas significativas para análisis de outliers.")
    print("Las variables numéricas existentes son años e IDs, que no aplican.")

## 7. Clustering

### 7.1 Preparación de datos

Para clustering usamos tanto variables numéricas como categóricas codificadas.

In [ ]:
# Seleccionar variables para clustering
# Excluir: municipios, meses (dominan el one-hot y crean clusters por mes),
# departamentos, y variables con >92% nulos
exclude_cluster = {'mesreg', 'mesocu', 'depreg', 'depocu', 'dredif', 'dnadif'}
cluster_cat = [c for c in cat_cols
               if 2 <= df[c].nunique() <= 15
               and df[c].notna().sum() > len(df) * 0.7
               and c not in exclude_cluster
               and 'mun' not in c and 'mup' not in c]

cluster_num = [c for c in meaningful_num if df[c].notna().sum() > len(df) * 0.7]

print(f"Variables para clustering:")
print(f"  Numéricas ({len(cluster_num)}):")
for c in cluster_num:
    print(f"    - {c}: {label(c)}")
print(f"  Categóricas ({len(cluster_cat)}, se codificarán con one-hot):")
for c in cluster_cat:
    print(f"    - {c}: {label(c)} ({df[c].nunique()} cat.)")

# Preparar DataFrame
df_clust = df[cluster_num + cluster_cat].dropna().copy()

# One-hot encode
if cluster_cat:
    df_encoded = pd.get_dummies(df_clust, columns=cluster_cat, drop_first=False)
else:
    df_encoded = df_clust.copy()

# Estandarizar
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_encoded)

# Muestra si es muy grande
if len(X_scaled) > 50000:
    np.random.seed(42)
    idx = np.random.choice(len(X_scaled), 50000, replace=False)
    X_sample = X_scaled[idx]
else:
    X_sample = X_scaled

print(f"\nDatos: {X_sample.shape[0]:,} filas x {X_sample.shape[1]} features")

### 7.2 Método del codo y silueta para determinar K óptimo

In [ ]:
K_range = range(2, 9)
inertias = []
silhouettes = []

sil_sample = X_sample[:10000] if len(X_sample) > 10000 else X_sample

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    km_labels = kmeans.fit_predict(X_sample)
    inertias.append(kmeans.inertia_)

    sil_labels = kmeans.predict(sil_sample)
    sil = silhouette_score(sil_sample, sil_labels)
    silhouettes.append(sil)
    print(f"  K={k}: inercia={kmeans.inertia_:,.0f}, silueta={sil:.4f}")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(K_range, inertias, 'bo-', linewidth=2)
ax1.set_xlabel('Número de clusters (K)')
ax1.set_ylabel('Inercia')
ax1.set_title('Método del Codo')

ax2.plot(K_range, silhouettes, 'ro-', linewidth=2)
ax2.set_xlabel('Número de clusters (K)')
ax2.set_ylabel('Coeficiente de Silueta')
ax2.set_title('Método de la Silueta')

plt.tight_layout()
plt.show()

best_k = list(K_range)[np.argmax(silhouettes)]
print(f"\nMejor K según silueta: {best_k} (score: {max(silhouettes):.4f})")

### 7.3 Clustering final e interpretación

In [ ]:
# Clustering con K óptimo
kmeans_final = KMeans(n_clusters=best_k, random_state=42, n_init=10)
cluster_labels = kmeans_final.fit_predict(X_scaled[:len(df_clust)])
df_clust['cluster'] = cluster_labels

print(f"Distribución de clusters (K={best_k}):")
cluster_dist = df_clust['cluster'].value_counts().sort_index()
for cl, count in cluster_dist.items():
    print(f"  Cluster {cl}: {count:>9,} ({count/len(df_clust)*100:.1f}%)")

In [ ]:
# Visualizar clusters con PCA (muestra para rendimiento)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled[:len(df_clust)])

plot_n = min(20000, len(X_pca))
np.random.seed(42)
plot_idx = np.random.choice(len(X_pca), plot_n, replace=False)

fig, ax = plt.subplots(figsize=(10, 8))
for cl in range(best_k):
    mask = cluster_labels[plot_idx] == cl
    ax.scatter(X_pca[plot_idx[mask], 0], X_pca[plot_idx[mask], 1],
              alpha=0.4, s=8, label=f'Cluster {cl}')

ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} varianza explicada)')
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} varianza explicada)')
ax.set_title(f'Clusters (K={best_k}) - Proyección PCA', fontsize=14)
ax.legend(loc='upper right', markerscale=3)
plt.tight_layout()
plt.show()

In [ ]:
# Interpretar clusters: perfil de cada cluster con variables categóricas
print("=== Perfil de cada cluster ===")
for cat_col in cluster_cat:
    print(f"\n--- {label(cat_col)} ({cat_col}) ---")
    ct = pd.crosstab(df_clust['cluster'], df_clust[cat_col], normalize='index') * 100
    print(ct.round(1).to_string())

In [ ]:
# Perfil numérico por cluster
if cluster_num:
    print("=== Medias de variables numéricas por cluster ===")
    cluster_means = df_clust.groupby('cluster')[cluster_num].mean()
    cluster_means.columns = [f"{c} ({label(c)})" for c in cluster_means.columns]
    display(cluster_means.round(2))

    # Boxplots por cluster
    n = len(cluster_num)
    if n > 0:
        ncols_plot = min(3, n)
        nrows_plot = (n + ncols_plot - 1) // ncols_plot
        fig, axes = plt.subplots(nrows_plot, ncols_plot, figsize=(5*ncols_plot, 4*nrows_plot))
        axes = np.array(axes).flatten() if n > 1 else [axes]

        for i, col in enumerate(cluster_num):
            df_clust.boxplot(column=col, by='cluster', ax=axes[i])
            axes[i].set_title(label(col))
            axes[i].set_xlabel('Cluster')

        for j in range(n, len(axes)):
            axes[j].set_visible(False)

        plt.suptitle('Variables numéricas por cluster', fontsize=14, y=1.02)
        plt.tight_layout()
        plt.show()